# libs

In [1]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from scipy import spatial
import glob
import os
import pandas as pd
import requests

# Data

## df pararius

In [2]:
list_of_files = glob.glob("data/temp/df_pararius_*.csv") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
df_pararius = pd.read_csv(latest_file, index_col=[0])

## zipcodeZ

In [3]:
df_zipcode = pd.read_csv('data/processed/zipcodeZ.csv',index_col=[0])

# What is new?
These are new postcodes that we do not have in our database

In [4]:
temp1 = df_pararius['postcode'].unique()
temp2 = df_zipcode['postcode'].unique()
cep_zip = list(set(temp1) - set(temp2))
print(len(cep_zip))

22


## crawler
get data from "postcode bij adres"

In [5]:
def html_content(cep):
    url = 'https://postcodebijadres.nl/' + cep
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"}
    soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")
    return soup

In [6]:
zipcode=pd.DataFrame({})
description_1={}

for cep in cep_zip:
    soup = html_content(cep)
    
    try:
        
        for num in range(9):
            split0 = soup.find_all('table',{"class":"table table-bordered mt-3"})[0]
            split1 = soup.find_all('table',{"class":"table table-bordered mt-3"})[1]
            
            split0_key = split0.find_all('th')[num].text
            split0_val = split0.find_all('td')[num].text

            lat = split1.find_all('td')[1].text
            lon = split1.find_all('td')[3].text
                                   
            description_1.update({
                split0_key:split0_val,
                'latitude':lat,
                'longitude':lon
            })

        zipcode_ = pd.DataFrame(description_1,index=[0])
        zipcode  = pd.concat([zipcode,zipcode_],0)

    except:
        print(cep)
        pass

<ipython-input-6-73aeada1214d>:26: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode  = pd.concat([zipcode,zipcode_],0)


3551WL
8316MB
5397CH
1036LZ
6224CV
4847GN
7773DG
4711SJ
5617BH
3931XZ
3431HM
6224CR
1314CD
3521VZ


## cleaning data and save

In [7]:
try:
    zipcode['Straat'] = zipcode['Straat'].str.replace('\n',"",regex=True)
    zipcode['Postcode'] = zipcode['Postcode'].str.replace(' ',"",regex=True)
    zipcode = zipcode.rename(columns={'Postcode':'postcode'})
    zipcode = pd.concat([zipcode,df_zipcode],0).reset_index(drop=True)
    zipcode.to_csv('data/processed/zipcodeZ.csv')
except:
    zipcode = df_zipcode.copy()

<ipython-input-7-d3fcb68d549f>:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode = pd.concat([zipcode,df_zipcode],0).reset_index(drop=True)


In [8]:
zipcode_pararius_coo = pd.merge(df_pararius, zipcode, how='left',on='postcode')

# organizing before saving

In [9]:
zipcode_pararius_coo = zipcode_pararius_coo.fillna(0).drop_duplicates(subset=['irl']).reset_index(drop=True)

In [10]:
zipcode_pararius_coo.loc[zipcode_pararius_coo['interior']==0, 'interior'] = 'Upholstered'
zipcode_pararius_coo.loc[zipcode_pararius_coo['status']==0, 'status'] = 'Free'

In [11]:
# zipcode_pararius_coo['city'] = zipcode_pararius_coo['irl'].str.split("/",expand=True)[2].str.replace("-", " ").str.title()

In [12]:
# prepare link on streamlit
zipcode_pararius_coo['url'] = 'https://www.pararius.com' + zipcode_pararius_coo['irl']
zipcode_pararius_coo['link'] = "<a target='_blank' href=" + (zipcode_pararius_coo['url']).astype(str) + ">" + (zipcode_pararius_coo['street']).astype(str) + "</a>"
zipcode_pararius_coo['img'] = "<a href=" + zipcode_pararius_coo['url'] + " target='blank'><img src=" + zipcode_pararius_coo['image'] + "title='rent' width='150' height='100'/></a>"

# save data

In [16]:
zipcode_pararius_coo.to_csv('app/df_coo_pararius.csv')
# zipcode_zipcode.to_csv('data/processed/zipcode.csv')

In [17]:
zipcode_pararius_coo#['city']

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,...,Postcodenummer,Buurt,Wijk,Plaats,Gemeente,Provincie,Straten,url,link,img
0,1550.0,1055 ND Amsterdam (Landlust),Apartment Admiraal De Ruijterweg,Hamilton Vastgoed,/apartment-for-rent/amsterdam/1e27decc/admiraa...,https://casco-media-prod.global.ssl.fastly.net...,60,Free,3,Upholstered,...,1055,Gibraltarbuurt,Landlust,Amsterdam,Amsterdam,Noord-Holland,\nHuninkhof 1 - 13 \n\nLangstraat 2 - 40 \n\n,https://www.pararius.com/apartment-for-rent/am...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...
1,1600.0,1055 ND Amsterdam (Landlust),Apartment Admiraal De Ruijterweg,Hamilton Vastgoed,/apartment-for-rent/amsterdam/fd962ef8/admiraa...,https://casco-media-prod.global.ssl.fastly.net...,65,Free,3,Upholstered,...,1055,Gibraltarbuurt,Landlust,Amsterdam,Amsterdam,Noord-Holland,\nHuninkhof 1 - 13 \n\nLangstraat 2 - 40 \n\n,https://www.pararius.com/apartment-for-rent/am...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...
2,950.0,2013 ZB Haarlem (Leidsebuurt-oost),Apartment Jan Nieuwenhuijzenstraat,Haarlem Real Estate,/apartment-for-rent/haarlem/17b48ca1/jan-nieuw...,https://media.pararius.nl/image/PR0001659000/P...,55,Free,2,Upholstered,...,2013,Leidsebuurt-oost,Zijlwegkwartier,Haarlem,Haarlem,Noord-Holland,0,https://www.pararius.com/apartment-for-rent/ha...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...
3,460.0,3565 BA Utrecht (Bedrijventerrein en omgeving),Studio Ontariodreef,Forest Property Management,/studio-for-rent/utrecht/1c3b229d/ontariodreef,https://media.pararius.nl/image/PR0001659000/P...,16,Free,1,Upholstered,...,3565,Bedrijventerrein en omgeving,Wijk 03 Overvecht,Utrecht,Utrecht,Utrecht,\nWesteinde 2 - 36 \n\nAssendelftstraat 2 - 2 ...,https://www.pararius.com/studio-for-rent/utrec...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/studio-for-re...
4,1325.0,2312 GW Leiden (Marewijk),Apartment Legewerfsteeg,RE/MAX Makelaarsgilde Leiden,/apartment-for-rent/leiden/0289e309/legewerfsteeg,https://media.pararius.nl/image/PR0001659000/P...,85,Free,3,Upholstered,...,2312,Marewijk,Binnenstad-Noord,Leiden,Leiden,Zuid-Holland,0,https://www.pararius.com/apartment-for-rent/le...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5511,1500.0,1505 TJ Zaandam (Bomenbuurt),House Lijsterbesstraat,Uw Partner in Onroerende Zaken,/house-for-rent/zaandam/1b23bcc5/lijsterbesstraat,https://casco-media-prod.global.ssl.fastly.net...,86,Under option,4,Upholstered,...,1505,Bomenbuurt,Wijk 11 Zaandam Zuid,Zaandam,Zaanstad,Noord-Holland,\nGasthuisstraat 1 - 25 \n\nBotersteeg 2 - 16 ...,https://www.pararius.com/house-for-rent/zaanda...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/house-for-ren...
5512,3450.0,3045 LJ Rotterdam (Zestienhoven),House Overschiese Kleiweg,BenHousing,/house-for-rent/rotterdam/e5d0ae4e/overschiese...,https://casco-media-prod.global.ssl.fastly.net...,294,Under option,6,Upholstered or furnished,...,3045,Zestienhoven,Overschie,Rotterdam,Rotterdam,Zuid-Holland,\nLeeuwendijk 1 - 1 \n\nMolendijk 6 - 32 \n\n,https://www.pararius.com/house-for-rent/rotter...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/house-for-ren...
5513,960.0,3762 GK Soest (Klaarwater),Apartment Wiardi Beckmanstraat,Regio Vastgoedmakelaars,/apartment-for-rent/soest/2e33b708/wiardi-beck...,https://casco-media-prod.global.ssl.fastly.net...,90,Under option,4,Shell,...,3762,Klaarwater,Wijk 02 Klaarwater,Soest,Soest,Utrecht,\nKorenpassage 5 - 43 \n\nKorenstraat 6 - 84 \n\n,https://www.pararius.com/apartment-for-rent/so...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.para